## Importando bibliotecas

In [70]:
# bibliotecas padrões
import pandas as pd
import numpy as np

# bibliotecas para NLP
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
from unidecode import unidecode
from sklearn.feature_extraction.text import TfidfVectorizer

# bibliotecas para modelagem
from sklearn.svm import SVC
from sklearn.metrics import precision_recall_fscore_support, classification_report
from sklearn.model_selection import KFold, cross_validate
from joblib import load

In [71]:
string_feedback = 'Gostei muito da palestra, mas achei demorado, os slides poderia ser mais rápido'
string_feedback = {'Feedback': string_feedback}
dados = pd.DataFrame({'Feedback': string_feedback})
dados

,Feedback
Feedback,Muito interessante


## Limpenza do texto

In [72]:
dados.Feedback = dados.Feedback.str.lower()
dados['Feedback'] = dados['Feedback'].apply(lambda x: unidecode(x))
dados.head()

,Feedback
Feedback,muito interessante


In [73]:
def RemovePunctuation(feedback):
    feedback = re.sub(r"[-|0-9]", "", feedback).lower()
    feedback = re.sub(r'[-./?!,":;()\']', ' ', feedback).lower()
    return (feedback)

lista_feedback = []
lista_feedback = [RemovePunctuation(feedback) for feedback in dados.Feedback]

dados.Feedback = lista_feedback
dados.head()

,Feedback
Feedback,muito interessante


In [74]:
dados['Feedback'] = dados['Feedback'].apply(lambda x: x.strip())
dados['Feedback'] = dados['Feedback'].apply(lambda x: x.replace("  ", " "))
dados.head()

,Feedback
Feedback,muito interessante


## Pré-Processamento

In [75]:
nlp = word_tokenize

In [76]:
dados['Contagem_palavras'] = dados['Feedback'].apply(lambda x: len(str(x).split(" ")))
dados.head()

,Feedback,Contagem_palavras
Feedback,muito interessante,2


In [77]:
def RemoveStopWords(feedback):
    palavras = [i for i in feedback.split() if not i in corr_palavras]
    return (" ".join(palavras))

In [78]:
lemmatizer = WordNetLemmatizer()
def Lemmatization(feedback):
  palavras = []
  for w in feedback.split():
    palavras.append(lemmatizer.lemmatize(w))
  return (" ".join(palavras))

In [79]:
#dados = dados[dados['Contagem_palavras'] > 2]

In [80]:
corr_palavras = pd.read_csv('palavras_corr.csv')
corr_palavras = list(corr_palavras.Palavras_corr.values)

In [81]:
lista_feedback = []
lista_feedback = [RemoveStopWords(feedback) for feedback in dados.Feedback]

In [82]:
dados['Feedback_palavras_irrelevantes'] = lista_feedback

In [83]:
# função para transformar o texto em Matrix Count e TFIDF

tfidf = load('vectorizer_tfidf.joblib')

def bow(feedbacks):
    bag_of_words_transformer = tfidf
    mx = bag_of_words_transformer.transform(feedbacks).todense()
    terms = bag_of_words_transformer.get_feature_names()
    dados_tfidf = pd.DataFrame(mx, columns=terms, index=feedbacks)
    
    return (dados_tfidf)

In [84]:
feedback_lem = dados['Feedback_palavras_irrelevantes'].apply(lambda x: Lemmatization(x))
dados_tfidf = bow(feedback_lem)
dados_tfidf.head()

,abordado,abordados,abordagem,acaba,acabei,acabou,academia,achando,achei,acho,...,video,vies,vimver,visao,visto,vivencias,volta,vou,vox,zoom
Feedback_palavras_irrelevantes,,,,,,,,,,,,,,,,,,,,,
muito,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Deploy do Modelo

In [85]:
svc = load('model_svc.joblib')

In [86]:
svc.predict(dados_tfidf)

array([1])

In [87]:
svc.predict_proba(dados_tfidf) * 100

array([[ 1.49041801, 98.50958199]])